<a href="https://colab.research.google.com/github/DataScienceAndEngineering/deep-learning-final-project-project-sidewalk/blob/rabiul/notebooks%20/Rabiul/Baseline_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction
** Please excute the following section:
1. 

### Loading Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt
from scipy import ndimage as nd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import pickle
from sklearn import metrics

In [ ]:
#loading data and model 
!gdown --id 1-3gbEPrVnD0YZ38ZixZrRjo3y2-NeOgr -O test.csv
!gdown --id 1TRDaaX13CqHE8xzIXKPnhLEhniWlUR9a -O train.csv
!gdown --id 1-14yL80KGLJzF2D6j3BXTH1dLa2knz_W -O model.sav

In [ ]:
model = pickle.load(open('/content/model.sav', 'rb'))

In [ ]:
X_train, X_test, y_train, y_test = train_test_data('/content/train.csv', '/content/test.csv')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
print(rf_accuracy(model, X_test,y_test))

In [ ]:
print(rf_accuracy(model, X_train,y_train))

# Data

###  Loading Data

In [ ]:
%%time
#downloading preprocessed data
!gdown --id 12rNfOiM4allOn8t4l0ErpvgmA9hEZPh2 -O processed_labels.zip
!gdown --id 145Ul0rdbQcX98lQz8IO5eOtI0ZlvKwz6 -O processed_images.zip

In [ ]:
%%time
#unzipping the preprocessed data to local dir
!unzip /content/processed_images.zip
!unzip /content/processed_labels.zip

### Pre-Processing Data Step 1

In [ ]:
#path to the directory for both image and label data 
img_path = '/content/content/processed_images/*'
lab_path = '/content/content/processed_labels/*'
#one sample data point
#/content/content/processed_images/aachen_000000_000019.png
#/content/content/processed_labels/aachen_000000_000019.png

In [ ]:
#loading the data indo a list, numpy array
%%time
train_images = glob.glob(img_path)
train_labels = glob.glob(lab_path)
print(len(train_images),len(train_labels))


In [ ]:
n=40
m=2310

In [ ]:
t = np.array(x_train['gray'].values)
t = t.reshape(int(t.shape[0]/40000),40000)
display_img(t[0].reshape(200,200))

In [ ]:
#test and train images 
x_train = create_train_image_df(train_images[:n])
x_test = create_test_image_df(train_images[2300:m])
print(x_train.shape,x_test.shape )
x_train.head()

In [ ]:
x_test.head(2)

In [ ]:
#Creating train mask image Data Frame
%%time
dim = (200,200)
y_train = pd.DataFrame()

for label_dir in train_labels[:n]:
  df2 = pd.DataFrame()
  lab = cv2.imread(label_dir, 0)
  #lab = cv2.resize(lab, dim).reshape(-1)
  df2['label'] = cv2.resize(lab, dim).reshape(-1) 
  y_train = pd.concat([y_train, df2])
print(y_train.shape)
y_train.head(2)

In [ ]:
#Creating test mask image Data Frame
%%time
dim = (200,200)
y_test = pd.DataFrame()

for label_dir in train_labels[2300:m]:
  df2 = pd.DataFrame()
  lab = cv2.imread(label_dir, 0)
  #lab = cv2.resize(lab, dim).reshape(-1)
  df2['label'] = cv2.resize(lab, dim).reshape(-1) 
  y_test = pd.concat([y_test, df2])
print(y_test.shape)
y_test.head(2)

In [ ]:
df_train = pd.concat([x_train, y_train], axis=1)
print(df_train.shape)
df_train.head(2)

In [ ]:
df_test = pd.concat([x_test, y_test], axis=1)
print(df_test.shape)
df_test.head(2)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
cd /content/drive/MyDrive/DL_Project

In [ ]:
#only execute this cell to save the preprocessed dataframe to google drive 
df_train.to_csv('df_train.csv', index=False)
df_test.to_csv('df_test.csv', index=False)
#Changing directory back to "Content"


In [ ]:
cd /content

## PreProcess Data Step 2

In [ ]:
#print(df_train.shape)
#print(df_test.shape)
print(df_train.shape, df_test.shape)

In [ ]:
#Execute this cell only if you want to use GPU and GPU is setup
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
#Execute this cell only if you want to remove all zero value where label = 0. 
df_train = df_train[df_train.label != 0]
df_test = df_test[df_test.label != 0]
print(df_train.shape, df_test.shape)

In [ ]:
X_Train = df_train.drop(labels = ["label"], axis=1)
Y_Train = df_train["label"].values

X_Test = df_test.drop(labels = ["label"], axis=1)
Y_Test = df_test["label"].values


print((X_Train.shape, Y_Train.shape),(X_Test.shape, Y_Test.shape))

In [ ]:
Y_Train = LabelEncoder().fit_transform(Y_Train)
Y_Test = LabelEncoder().fit_transform(Y_Test)
print(Y_Train.shape, Y_Test.shape)

#Feature Importance 

# MODEL

In [ ]:
model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            bootstrap=True, oob_score=False, n_jobs=None, random_state=2, 
                            verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, 
                            max_samples=None)

## FIT & Save Model

In [ ]:
cd /content/drive/MyDrive/DL_Project

In [ ]:
%%time
## Train the model on training data
model.fit(X_Train, Y_Train)

In [ ]:
#Save the trained model as pickle string to google drive for future use 
filename = "rf_model2_withzero.sav"
pickle.dump(model, open(filename, 'wb'))
#Changing directory back to "Content"
cd /content

## Predict

In [ ]:
prediction_test = model.predict(X_Test)

In [ ]:
print ("Testing Accuracy = ", metrics.accuracy_score(Y_Test, prediction_test))

In [ ]:
Y_pred = prediction_test.reshape(int(prediction_test.shape[0]/40000),40000)
Y_true = Y_Test.reshape(int(Y_Test.shape[0]/40000),40000)
print(Y_true.shape,Y_pred.shape )

In [ ]:
print ("Testing Accuracy = ", metrics.accuracy_score(Y_true[0].reshape(200,200), Y_pred[0].reshape(200,200)))

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

In [ ]:
print ("Training Accuracy = ", jaccard_score(Y_true, Y_pred,average='micro'))

# Functions

In [ ]:
def gabor_feature_extraction(img, lamda, gamma):
  kernel = cv2.getGaborKernel((8, 8), 1, 0, lamda, gamma, 0, ktype=cv2.CV_32F)
  gabor_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)
  return gabor_img.reshape(-1,1)/255


In [ ]:
def preprocess_image(img):
  #function will accept a numpy.ndarray

  #GAUSSIAN blur 1
  gblur = cv2.GaussianBlur(img, (5,5),7)
  #Laplacian
  laplacian = cv2.Laplacian(img, cv2.CV_64F, ksize=7) 
  #SOBEL
  sobele = cv2.Sobel(img,cv2.CV_64F, 0,1,ksize=5)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)

  #return cany
  return (gblur.reshape(-1)/255,laplacian.reshape(-1)/255,sobele.reshape(-1)/255,cany.reshape(-1)/255)

In [ ]:
def preprocess_image_org(img):
  #GAUSSIAN blur 1
  blur1 = nd.gaussian_filter(img, sigma=3)
  #GAUSSIAN blur2
  blur2 = nd.gaussian_filter(img, sigma=7)
  #SOBEL
  sobele = sobel(blur2)
  #CANNY 
  cany = cv2.Canny(np.uint8(sobele), 100,200)
  #return cany
  return (blur1.reshape(-1,1),blur2.reshape(-1,1),sobele.reshape(-1,1),cany.reshape(-1,1))

In [ ]:
def display_img(img):
  fig = plt.figure(figsize=(8,8))
  ax = fig.add_subplot(111)
  ax.imshow(img,cmap='gray')

In [ ]:
def load_img(image_path):
  return cv2.imread(image_path, 0).astype(np.float32)/255


In [ ]:
def generate_one_test_image(image_dir):
  img = cv2.imread(image_dir, 0)
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)
  return np.hstack((g1.reshape(-1,1), binay_inv.reshape(-1,1), adaptive_thresh.reshape(-1,1), 
                    gb1.reshape(-1,1), gb2.reshape(-1,1), blur.reshape(-1,1), 
                    laplacian.reshape(-1,1), sobel.reshape(-1,1), cany.reshape(-1,1)))


In [ ]:
def generate_one_test_image_df(image_dir):
  dim = (200,200)
  test_df = pd.DataFrame()
  img = cv2.imread(image_dir, 0)
  img = cv2.resize(img, dim)
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)

  test_df['gray']= g1.reshape(-1)
  test_df['binay_inv']= binay_inv.reshape(-1)
  test_df['adaptive_thresh']=adaptive_thresh.reshape(-1)
  test_df['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
  test_df['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
  test_df['blur'], test_df['laplacian'], test_df['sobelx'], test_df['canny'] = blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1), cany.reshape(-1)

  return test_df


In [ ]:
def generate_one_test_image_df_a(image_dir):
  #this function is getting an image as an input 
  test_df = pd.DataFrame()
  #img = cv2.imread(image_dir, 0)
  img = image_dir
  g1 = img/255
  _,thresh3  = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY_INV)
  binay_inv = thresh3/255
  adaptive_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11, 11)/255
  gb1 = gabor_feature_extraction(img,.8, .05 )/255
  gb2 = gabor_feature_extraction(img,1.6, .5 )/255
  blur, laplacian, sobel, cany = preprocess_image(img)

  test_df['gray']= g1.reshape(-1)
  test_df['binay_inv']= binay_inv.reshape(-1)
  test_df['adaptive_thresh']=adaptive_thresh.reshape(-1)
  test_df['gabor1'] = gabor_feature_extraction(img,.8, .05 ).reshape(-1)
  test_df['gabor2'] = gabor_feature_extraction(img,1.6, .5 ).reshape(-1)
  test_df['blur'], test_df['laplacian'], test_df['sobelx'], test_df['canny'] = blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1), cany.reshape(-1)

  return test_df

In [ ]:
def visualize_images(img, gt, pred):
    if pred is not None:
        fig, axes = plt.subplots(1, 3, figsize=(12, 8))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(img)
    axes[0].set_title('Actual Image')

    axes[1].imshow(gt)
    axes[1].set_title('Masked Image')
    
    if pred is not None:
        axes[2].imshow(pred)
        axes[2].set_title('Predicted Image')

In [ ]:
def visualize_images_2(img, gt, pred):
    if pred is not None:
        fig, axes = plt.subplots(1, 3, figsize=(12, 8))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(img)
    axes[0].set_title('Actual Image')

    axes[1].imshow(gt)
    axes[1].set_title('erosion5')
    
    if pred is not None:
        axes[2].imshow(pred)
        axes[2].set_title('Gradiant')

In [ ]:
def plot(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1,len(acc)+1)

  plt.title('Training and validation accuracy')
  plt.plot(epochs, acc, color='blue', label='Train')
  plt.plot(epochs, val_acc, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  _ = plt.figure()
  plt.title('Training and validation loss')
  plt.plot(epochs, loss, color='blue', label='Train')
  plt.plot(epochs, val_loss, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()

In [ ]:
def create_test_image_df(train_images,n):
  '''provide a list of image directory and n= number of image'''
  image_df = pd.DataFrame()
  for img_dir in train_images[:n]:
    image_df = pd.concat([image_df, generate_one_test_image_df(img_dir)])
  return image_df

In [ ]:
def create_train_image_df(train_images):
  '''provide a list of image directory and n= number of image you want , test image starts from 2300'''
  image_df = pd.DataFrame()
  for img_dir in train_images:
    image_df = pd.concat([image_df, generate_one_test_image_df(img_dir)])
  print('train', image_df.shape)
  return image_df

def create_test_image_df(train_images):
  import pandas as pd
  '''provide a list of image directory and n= number of image'''
  image_df = pd.DataFrame()
  for img_dir in train_images:
    single_img_df = generate_one_test_image_df(img_dir)
    image_df = pd.concat([image_df,single_img_df ])
  print('test', image_df.shape)
  return image_df

In [ ]:
def train_test_data(df_train, df_test):
  '''df_train, df_test > provide both test and train df with image data and label data'''
  df_train = pd.read_csv(df_train) 
  df_test = pd.read_csv(df_test) 
  X_Train = df_train.drop(labels = ["label"], axis=1)
  Y_Train = df_train["label"].values

  X_Test = df_test.drop(labels = ["label"], axis=1)
  Y_Test = df_test["label"].values

  Y_Train = LabelEncoder().fit_transform(Y_Train)
  Y_Test = LabelEncoder().fit_transform(Y_Test)

  return X_Train,X_Test, Y_Train, Y_Test

In [ ]:
def rf_accuracy(model, X, Y):
  'provide the randomforest model, X: test/train data, y: test/train label '
  prediction_test = model.predict(X)
  Y_pred = prediction_test.reshape(int(prediction_test.shape[0]/40000),40000)
  Y_true = Y.reshape(int(Y.shape[0]/40000),40000)
  return metrics.accuracy_score(Y_true[0].reshape(200,200), Y_pred[0].reshape(200,200))

In [ ]:
def plot_feature_importance(importance,names,model_type):
  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(10,8))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

In [ ]:
def precision_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean(intersect/total_pixel_pred)
    return round(precision, 3)

def recall_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean(intersect/total_pixel_truth)
    return round(recall, 3)


def accuracy(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    xor = np.sum(groundtruth_mask==pred_mask)
    acc = np.mean(xor/(union + xor - intersect))
    return round(acc, 3)

def dice_coef(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_sum = np.sum(pred_mask) + np.sum(groundtruth_mask)
    dice = np.mean(2*intersect/total_sum)
    return round(dice, 3) #round up to 3 decimal places

def iou(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    iou = np.mean(intersect/union)
    return round(iou, 3)

# Edge detection and  Thresholding

In [ ]:
mask_img = cv2.imread('/content/content/processed_labels/strasbourg_000001_038281.png',0)
cl_img =cv2.imread('/content/content/processed_images/strasbourg_000001_038281.png')
img = cv2.imread('/content/content/processed_images/strasbourg_000001_038281.png',0)

In [ ]:
img = cv2.imread(train_images[0],0)

In [ ]:
display_img(img)

In [ ]:
_,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
display_img(thresh1)

In [ ]:
_,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
plt.imshow(thresh2,cmap='gray')

In [ ]:
_,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
plt.imshow(thresh3,cmap='gray')

In [ ]:
_,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
plt.imshow(thresh4,cmap='gray')

In [ ]:
_,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)
plt.imshow(thresh5,cmap='gray')

In [ ]:
gamma = 1/4
effected_image1 = np.power(img, gamma)
display_img(effected_image1)

In [ ]:
gamma = 2
effected_image2 = np.power(img, gamma)
display_img(effected_image2)

In [ ]:
median_blur = cv2.medianBlur(img,5)
display_img(median_blur)

In [ ]:
#Erosion
#Erodes away boundaries of foreground objects. Works best when foreground is light color (preferrably white) and background is dark.
kernel = np.ones((5,5),np.uint8)
erosion1 = cv2.erode(img,kernel,iterations = 1)
display_img(erosion1)

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion5 = cv2.erode(img,kernel,iterations = 4)
display_img(erosion5)

In [ ]:
white_noise = np.random.randint(low=0,high=2,size=(256,512))
white_noise = white_noise*255
noise_img = white_noise+img
display_img(noise_img)

In [ ]:
gradient = cv2.morphologyEx(img,cv2.MORPH_GRADIENT,kernel)
display_img(gradient)

In [ ]:
eq_img= cv2.equalizeHist(img)
display_img(eq_img)

In [ ]:
kernel = cv2.getGaborKernel((8, 8), 1, 0, .8, .05, 0, ktype=cv2.CV_32F)
gabor1 = cv2.filter2D(img, cv2.CV_8UC3, kernel)
plt.imshow(gabor1,cmap='gray')

In [ ]:
kernel2 = cv2.getGaborKernel((8, 8), 1, 0, 1.6, .5, 0, ktype=cv2.CV_32F)
gabor2 = cv2.filter2D(img, cv2.CV_8UC3, kernel2)
plt.imshow(gabor2,cmap='gray')

In [ ]:
# kernel size
ksize = 31  
# sigma for Gaussian envelope
sigma = 5  
# range of orientation values
theta_range = np.arange(0, np.pi, np.pi / 4) 
# frequency of sinusoidal wave
frequency = 0.3  
# phase of sinusoidal wave
phase = 0  
  
# for loop for generating different rotation
for theta in theta_range:
    kernel = cv2.getGaborKernel((ksize, ksize),
                                sigma, theta,
                                frequency, phase)
    gabor_img = cv2.filter2D(img, cv2.CV_8UC3, kernel)
    plt.imshow(gabor_img,
               cmap='gray')
    plt.title("Kernel with orientation = " + str(theta))
    plt.show()

In [ ]:
cl_img =cv2.imread('/content/content/processed_images/strasbourg_000001_038281.png')
 # Extract RGB color features
r, g, b = cv2.split(cl_img)
plt.imshow(r)

In [ ]:
plt.imshow(g)

In [ ]:
plt.imshow(b)

In [ ]:
train_images[0], train_images[0]

In [ ]:
cl_img =cv2.imread('/content/content/processed_images/strasbourg_000001_038281.png')
img = cv2.imread('/content/content/processed_images/strasbourg_000001_038281.png',0)
 # Extract RGB color features
r, g, b = cv2.split(cl_img)

In [ ]:
cl_img =cv2.imread(train_images[0])
img = cv2.imread(train_images[0],0)
 # Extract RGB color features
r, g, b = cv2.split(cl_img)

In [ ]:
_,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
_,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
_,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
_,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
_,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)
effected_image1 = np.power(img, .25)
effected_image2 = np.power(img, 2)
median_blur = cv2.medianBlur(img,5)
#Erosion
kernel = np.ones((5,5),np.uint8)
erosion1 = cv2.erode(img,kernel,iterations = 1)
erosion5 = cv2.erode(img,kernel,iterations = 4)
white_noise = np.random.randint(low=0,high=2,size=(256,512))
white_noise = white_noise*255
noise_img = white_noise+img
gradient = cv2.morphologyEx(img,cv2.MORPH_GRADIENT,kernel)
eq_img= cv2.equalizeHist(img)
kernel = cv2.getGaborKernel((8, 8), 1, 0, .8, .05, 0, ktype=cv2.CV_32F)
gabor1 = cv2.filter2D(img, cv2.CV_8UC3, kernel)
kernel2 = cv2.getGaborKernel((8, 8), 1, 0, 1.6, .5, 0, ktype=cv2.CV_32F)
gabor2 = cv2.filter2D(img, cv2.CV_8UC3, kernel2)

## feature importance

In [ ]:
df = pd.DataFrame()
df['gray'] = img.reshape(-1)/255

In [ ]:
df['r'] = r.reshape(-1)/255
df['g'] = g.reshape(-1)/255
df['b'] = b.reshape(-1)/255

In [ ]:
df['thresh1'] = thresh1.reshape(-1)/255
df['thresh2'] = thresh2.reshape(-1)/255
df['thresh3'] = thresh3.reshape(-1)/255
df['thresh4'] = thresh4.reshape(-1)/255
df['thresh5'] = thresh5.reshape(-1)/255

In [ ]:
df['effected_image2'] = effected_image2.reshape(-1)/255
df['effected_image1'] = effected_image1.reshape(-1)/255

In [ ]:
df['median_blur'] = median_blur.reshape(-1)/255
df['erosion1'] = erosion1.reshape(-1)/255
df['erosion5'] = erosion5.reshape(-1)/255
df['noise_img'] = noise_img.reshape(-1)/255
df['gradient'] = gradient.reshape(-1)/255
df['eq_img'] = eq_img.reshape(-1)/255
df['gabor1'] = gabor1.reshape(-1)/255
df['gabor2'] = gabor2.reshape(-1)/255

In [ ]:
 blur, laplacian, sobel, cany = preprocess_image(img)
df['blur'], df['laplacian'], df['sobelx'], df['canny'] = blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1), cany.reshape(-1)

In [ ]:
df.shape

In [ ]:
train_labels[0]

In [ ]:
mask_img = cv2.imread('/content/content/processed_labels/strasbourg_000001_038281.png',0)
plt.imshow(mask_img)

In [ ]:
mask_img = cv2.imread(train_labels[0],0)
plt.imshow(mask_img)

In [ ]:
df['Labels'] = mask_img.reshape(-1)/255

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = df["Labels"].values
Y = LabelEncoder().fit_transform(Y)

In [ ]:
#removing label data 
X = df.drop(labels = ["Labels"], axis=1) 

In [ ]:
model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, 
                            min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                            max_leaf_nodes=None, min_impurity_decrease=0.0, 
                            bootstrap=True, oob_score=False, n_jobs=None, random_state=2, 
                            verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, 
                            max_samples=None)

In [ ]:
# Train the model on training data
#model.fit(X_train, y_train)
model.fit(X, Y)

In [ ]:
feature_list = list(X.columns)
feature_imp = pd.Series(model.feature_importances_,index=feature_list).sort_values(ascending=False)
print(feature_imp)

In [ ]:
plot_feature_importance(model.feature_importances_,X.columns,'RANDOM FOREST ')

In [ ]:
features = feature_imp[feature_imp>.045]
features

In [ ]:
plot_feature_importance(features ,features.index,'RANDOM FOREST ' )

note: 
/content/content/processed_images/hanover_000000_029455.png /content/content/processed_labels/hanover_000000_029455.png
content/content/processed_images/strasbourg_000000_015602.png /content/content/processed_labels/strasbourg_000000_015602.png

# GridSearch CV

## Data

In [ ]:
n=17
train_images[n],train_labels[n]

In [ ]:
n=17
m = cv2.imread(train_images[n]
l = train_labels[n]

In [ ]:
#Test data 
cl_img =cv2.imread('/content/content/processed_images/strasbourg_000000_015602.png')
img = cv2.imread('/content/content/processed_images/strasbourg_000000_015602.png',0)
 # Extract RGB color features

In [ ]:
#training data 
cl_img =cv2.imread('/content/content/processed_images/hanover_000000_029455.png')
img = cv2.imread('/content/content/processed_images/hanover_000000_029455.png',0)
 # Extract RGB color features


In [ ]:
#img = cv2.imread('content/content/processed_images/strasbourg_000000_015602.png',0)
plt.imshow(img)

In [ ]:
kernel = np.ones((5,5),np.uint8)
kernel2 = cv2.getGaborKernel((8, 8), 1, 0, 1.6, .5, 0, ktype=cv2.CV_32F)
erosion5 = cv2.erode(img,kernel,iterations = 4)
gradient = cv2.morphologyEx(img,cv2.MORPH_GRADIENT,kernel)
blur, laplacian, sobel, _ = preprocess_image(img)
median_blur = cv2.medianBlur(img,5)
erosion1 = cv2.erode(img,kernel,iterations = 1)
r, _, _ = cv2.split(cl_img)
gabor2 = cv2.filter2D(img, cv2.CV_8UC3, kernel2)
_,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

In [ ]:
df_gridcv = pd.DataFrame()

In [ ]:
df_gridcv['erosion5'] = erosion5.reshape(-1)/255
df_gridcv['gradient'] = gradient.reshape(-1)/255
df_gridcv['blur'], df_gridcv['laplacian'], df_gridcv['sobelx']= blur.reshape(-1), laplacian.reshape(-1), sobel.reshape(-1)
df_gridcv['median_blur'] = median_blur.reshape(-1)/255
df_gridcv['erosion1'] = erosion1.reshape(-1)/255
df_gridcv['r'] = r.reshape(-1)/255
df_gridcv['gabor2'] = gabor2.reshape(-1)/255
df_gridcv['thresh5'] = thresh5.reshape(-1)/255
# Test    '/content/content/processed_labels/strasbourg_000000_015602.png'
# Train   '/content/content/processed_labels/hanover_000000_029455.png'
mask_img = cv2.imread('/content/content/processed_labels/strasbourg_000000_015602.png',0)
df_gridcv['Labels'] = mask_img.reshape(-1)/255

In [ ]:
#Test ONly
from sklearn.preprocessing import LabelEncoder
Y_test = df_gridcv["Labels"].values
Y_est = LabelEncoder().fit_transform(Y)
#removing label data 
X_test= df_gridcv.drop(labels = ["Labels"], axis=1) 
# X_test, Y_est

In [ ]:
from sklearn.preprocessing import LabelEncoder
Y = df_gridcv["Labels"].values
Y = LabelEncoder().fit_transform(Y)
#removing label data 
X = df_gridcv.drop(labels = ["Labels"], axis=1) 

In [ ]:
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, cross_val_score, GridSearchCV, RandomizedSearchCV
#GridSearch & Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

In [ ]:
rf = RandomForestClassifier()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf_grid_cv = GridSearchCV(estimator = rf, param_grid = random_grid,
                          cv = 5, verbose=2,n_jobs = 8,scoring= 'accuracy')

In [ ]:
X = [1,2,3,4,5]
Y= [1,2,3,4,5]

In [ ]:
%time rf_grid_cv.fit(X, Y)
print(rf_grid_cv.best_params_)

In [ ]:
print(rf_grid_cv.best_params_)

In [ ]:
g = {'n_estimators': [100, 200, 400],
     'max_depth': [40, 50, 70],
     'min_samples_split': [5,10,20],
     'min_samples_leaf': [ 20, 40,60]}

In [ ]:
rf_grid_cv = GridSearchCV(estimator = rf, param_grid = g,
                          cv = 5, verbose=2,n_jobs = 8,scoring= 'accuracy')

In [ ]:
%time rf_grid_cv.fit(X, Y)
print(rf_grid_cv.best_params_)

In [ ]:
print(rf_grid_cv.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, criterion='gini',
                           max_depth=None, n_jobs=-1, random_state=42)
#n_estimators=2000,max_features='gini',random_state=101, n_jobs=-1,max_depth=60, min_samples_leaf=1, min_samples_split=2)

In [ ]:
rf.fit(X, Y)

In [ ]:
filename = "rf_model2_10feature.sav"
pickle.dump(rf, open(filename, 'wb'))

In [ ]:
prediction_test = rf.predict(X)

In [ ]:
prediction_test.shape

In [ ]:
Y_pred = prediction_test.reshape(256,512)
Y_true = Y.reshape(256,512)

In [ ]:
display_img(Y_pred)

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

In [ ]:
print ("Training Accuracy = ", jaccard_score(Y_true, Y_pred,average='micro'))

In [ ]:
Y.shape.reshape(256,512)

In [ ]:
#X_test, Y_est
prediction_test = rf.predict(X_test)

In [ ]:
Y_pred = prediction_test.reshape(256,512)
Y_true = Y_est.reshape(256,512)

In [ ]:
print ("Training Accuracy = ", jaccard_score(Y_true, Y_pred,average='micro'))

In [ ]:
plt.imshow(Y_pred)

In [ ]:
plt.imshow(Y_true)

In [ ]:
print ("Testing Accuracy = ", metrics.accuracy_score(Y_true, Y_pred))

In [ ]:
visualize_images(cl_img, Y_true,Y_pred )

In [ ]:
precision_score_(Y_true,Y_pred)

In [ ]:
recall_score_(Y_true,Y_pred)

In [ ]:
accuracy(Y_true,Y_pred)

In [ ]:
dice_coef(Y_true,Y_pred)

In [ ]:
iou(Y_true,Y_pred)